# Procesamientos de imagenes y video

La pr&aacute;ctica requiri&oacute; grabar a una persona al volante con el motor apagado, a la altura de los ojos. El conductor debe alternar su mirada hacia el frente, izquierda, derecha, retrovisor y palanca.
Este proyecto tiene la finalidad de evaluar nuevos videos en tiempo real para determinar el desempeno del detector de PAVs.

In [10]:
# Importando librerias
import pandas as pd
import cv2
import os
import mediapipe as mp

In [3]:
Ruta = "./Imagen/"
dir_list = os.listdir(Ruta)
dir_list.sort()
lista_csv = []
for i in dir_list:
    fnames = os.listdir(Ruta + i)
    for f in fnames:
        if f.endswith(".csv"):
            df = pd.read_csv(Ruta + i + f"/{f}")
            name = f.replace('csv', 'mp4')
            df['source'] = Ruta + i + f"/{name}"
            df['subject'] = name[:-4]
            lista_csv.append(df)

In [4]:
# Dataframe con formato diferente
df_last = lista_csv[-1]
dfsCsv = lista_csv[:-1]

In [5]:
# Definir el mapeo de valores
mapeo_valores = {
    'frente': 'FRENTE',
    'izquierdo': 'IZQUIERDA',
    'central': 'RETROVISOR',
    'derecho': 'DERECHA',
    'palanca': 'PALANCA'
}

# Mapear los valores de la columna 'label'
df_last['label'] = df_last['label'].map(mapeo_valores)

df_last = df_last[['inicio', 'fin', 'label', 'source', 'subject']]
df_last

,inicio,fin,label,source,subject
0,00:00,00:12,FRENTE,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
1,00:13,00:24,IZQUIERDA,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
2,00:25,00:35,RETROVISOR,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
3,00:37,00:47,DERECHA,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
4,00:48,01:00,PALANCA,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT


In [6]:
dfs_concat = []
dfs_concat.append(df_last)
for df in dfsCsv:
    source = df['source'][0]
    subject = df['subject'][0]
    columns_to_keep = ['FRENTE', 'RETROVISOR', 'IZQUIERDA', 'DERECHA', 'PALANCA']
    df = df[columns_to_keep]

    # Usar melt para cambiar el formato del DataFrame
    df = df.melt(var_name='label', value_name='intervalo')

    # Separar la columna 'intervalo' en 'inicio' y 'fin'
    df[['inicio', 'fin']] = df['intervalo'].str.split('-', expand=True)

    # Eliminar la columna 'intervalo'
    df.drop(columns='intervalo', inplace=True)

    # Reordenar las columnas
    df = df[['inicio', 'fin', 'label']]

    df['source'] = source
    df['subject'] = subject
    df = df.dropna(axis=0)
    dfs_concat.append(df)

In [7]:
IMG_df = pd.concat(dfs_concat)

In [14]:
# Crear la carpeta para guardar los videos recortados
output_folder = './sources'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterar sobre cada fila del DataFrame
for index, row in IMG_df.iterrows():
    subject = row['subject']
    inicio = row['inicio']
    fin = row['fin']
    label = row['label']
    source = row['source']
    
    # Cargar el video
    video_capture = cv2.VideoCapture(source)
    
    # Obtener la tasa de fotogramas del video
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    
    # Verificar que el formato de inicio y fin sea válido
    if ':' not in inicio or ':' not in fin:
        print(f"Error en el formato de tiempo en la fila {index}. Saltando esta fila.")
        continue
    
    # Convertir los tiempos de inicio y fin a segundos
    inicio_segundos = int(inicio.split(':')[0]) * 60 + int(inicio.split(':')[1])
    fin_segundos = int(fin.split(':')[0]) * 60 + int(fin.split(':')[1])
    
    # Crear el nombre del archivo de salida
    folder_PAV = f'./sources/{label}'
    if not os.path.exists(folder_PAV):
        os.makedirs(folder_PAV)
    output_filename = f"{folder_PAV}/{subject}_{index}.mp4"
    IMG_df.at[index, 'source'] = output_filename
    
    # Crear un objeto VideoWriter para guardar el video recortado
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_filename, fourcc, fps, (int(video_capture.get(3)), int(video_capture.get(4))))
    
    # Establecer el cuadro actual en el tiempo de inicio
    video_capture.set(cv2.CAP_PROP_POS_FRAMES, inicio_segundos * fps)
    
    # Leer y escribir cada cuadro hasta el tiempo de finalización
    while video_capture.get(cv2.CAP_PROP_POS_FRAMES) < fin_segundos * fps:
        ret, frame = video_capture.read()
        if ret:
            video_writer.write(frame)
        else:
            break
    
    # Liberar los recursos
    video_capture.release()
    video_writer.release()

print("Videos guardados en la carpeta 'sources'.")

Videos guardados en la carpeta 'sources'.


In [19]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_point = []
    # Keypoint
    for point in landmarks:
        landmark_x = min(int(point.x * image_width), image_width - 1)
        landmark_y = min(int(point.y * image_height), image_height - 1)
        
        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [20]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list